# Notebook to run classification on ESC50 or UrbanSound8K datasets. Input are spectrogram images transformed from the audio samples.  Look at DataPrep folder for functions to prepare the spectrograms

### First we load the images then order them into numpy arrays

In [1]:
%matplotlib notebook

import numpy as np
import os
import re
import tensorflow as tf
from scipy import ndimage
import time
from datetime import datetime
import matplotlib.pyplot as plt
import random
from params import *
import spectreader

import model_single2 as m #import single-task learning CNN model

#np.set_printoptions(threshold=np.nan)  #if want to see full output of big arrays in Jupyter notebook

In [2]:
def time_taken(elapsed):
    """To format time taken in hh:mm:ss. Use with time.monotic()"""
    m, s = divmod(elapsed, 60)
    h, m = divmod(m, 60)
    return "%d:%02d:%02d" % (h, m, s)

def get_subdirs(a_dir):
    """ Returns a list of sub directory names in a_dir""" 
    return [name for name in os.listdir(a_dir)
            if (os.path.isdir(os.path.join(a_dir, name)) and not (name.startswith('.')))]

def esc50get_fold(string):
    """get fold no. from ESC-50 dataset using the filename. Labels #1-5"""
    label_pos = string.index("-")
    return string[label_pos-1]

def us8kget_fold(string):
    """use to grab fold no. of the UrbanSound8K dataset 
    e.g. filename: ../13230-0-0-1__fold3.tif
    we want to get the number 3 after 'fold' which tells us the correct fold number""" 
    try:
        found = re.search('__fold(.+?).tif', string).group(1)
        return found
    except AttributeError:
        print("No matched string found.")
        return None

def get_train_test(data,test_proportion,seed):
    """Splits data into arrays of test and training samples according to test_proportion
    Can use train_test_split from sklearn.model_selection library for similar functionality 
    """  
    random.seed(seed)
    test_samples = random.sample(data, int(len(data) * test_proportion)) #randomly pick out test samples
    train_samples = list(set(data) - set(test_samples)) #the compliment will be used for training
    return train_samples, test_samples

def fetch_files_kfold(spec_dir,dataset):
    """Returns lists of full pathnames of datafiles and their respective (numerical) labels and folds
    dataset: "US8K" or "ESC50"
    """
    filepaths = []
    labels = []
    folds = []
    i = 0
    class_folders = get_subdirs(spec_dir)
    for folder in class_folders:
        class_files = os.listdir(spec_dir + "/" + folder)
        if dataset == "US8K":
            fold = [int(us8kget_fold(fp)) for fp in class_files]
        elif dataset == "ESC50":
            fold = [int(esc50get_fold(fp)) for fp in class_files]
        else:
            raise ValueError("please only enter 'US8K' or 'ESC50'")
        class_files = [spec_dir + "/" + folder + "/" + fp for fp in class_files]
        class_labels = [i] * len(class_files)
        #print('Distribution of folds in class', i, ':', c.Counter(fold))
        
        folds.extend(fold)
        filepaths.extend(class_files)
        labels.extend(class_labels)
        i += 1
    return filepaths, labels, folds

def fetch_files_holdout(spec_dir,hold_prop,seed):
    """Returns lists of full pathnames of datafiles and their respective (numerical) labels, split to training and test sets
    hold_prop: proportion between 0-1 to hold out as test set in each class
    seed: random seed for pseudorandom number generator. Use same seed to maintain same test set.
    """
    from sklearn.model_selection import train_test_split
    
    train_filepaths = []
    test_filepaths = []
    train_labels = []
    test_labels = []
    i = 0
    class_folders = get_subdirs(spec_dir)
    
    for folder in class_folders:
        class_files = os.listdir(spec_dir + "/" + folder)
        class_files = [spec_dir + "/" + folder + "/" + fp for fp in class_files]
        class_labels = [i] * len(class_files)

        train_files, test_files, train_label, test_label = train_test_split(class_files, class_labels,
                                                                              test_size=hold_prop, random_state=seed)
        train_filepaths.extend(train_files)
        test_filepaths.extend(test_files)
        train_labels.extend(train_label)
        test_labels.extend(test_label)
        i += 1
    return train_filepaths,test_filepaths,train_labels,test_labels

def fetch_files(spec_dir):
    """Returns lists of full pathnames of datafiles and their respective (numerical) labels"""
    filepaths = []
    labels = []
    i = 0
    class_folders = get_subdirs(spec_dir)
    for folder in class_folders:
        class_files = os.listdir(spec_dir + "/" + folder)
        class_files = [spec_dir + "/" + folder + "/" + fp for fp in class_files]
        class_labels = [i] * len(class_files)
        
        filepaths.extend(class_files)
        labels.extend(class_labels)
        i += 1
    return filepaths, labels

In [ ]:
# firstly retrieve all the filenames, labels and folds
import collections as c

filepaths,labels,folds = fetch_files_kfold(STFT_dataset_path,dataset_name)
print(c.Counter(folds))

In [3]:
def shuffle(dataset, labels):
    """Randomizes order of elements in input arrays"""
    import random
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:,:]
    shuffled_labels = labels[permutation,:]
    return shuffled_dataset, shuffled_labels

def read_image(data):
    """For an array of filenames load each image. Returns a numpy array"""
    images = np.array([np.array(ndimage.imread(fp).astype(float)) - 0.5 for fp in data])
    return images
    
def sort_by_fold(folds,images,labels):
    """Sort images and labels in the order of folds
    folds: the 'folds' output of fetch_files_kfold
    * no longer have to sort by fold since the correct fold is picked during training time
    """
    fold_order = np.argsort(folds, kind="mergesort")
    sorted_folds = np.array(folds)[fold_order]
    sorted_images = images[fold_order] 
    sorted_labels = np.array(labels)[fold_order]
    return sorted_images, sorted_labels, sorted_folds

def reformat(dataset, labels, n_labels):
    """Reformats to appropriate shape for tensorflow covnet. Use 1-hot encoding for labels"""
    dataset = dataset.reshape((-1, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS)).astype(np.float32)
    labels = (np.arange(n_labels) == labels[:,None]).astype(np.int32)
    return dataset, labels

In [ ]:
#read in images, and reformat to Tensorflow-friendly array shapes

all_images = read_image(filepaths)
all_labels = np.array(labels)
folds = np.array(folds)
images, labels = reformat(all_images, all_labels, N_LABELS)
print('Dataset',images.shape, labels.shape)

In [ ]:
#functions to save/load numpy arrays to/from file

def save_sets(sets,name):
    """Writes the data array to .npy file. Can be loaded using load_set.
    sets: arrays to be saved. can take a list
    name: string to name the file. follow same order as in sets 
    """ 
    ind = 0
    for x in sets:
        np.save(save_path + '/{}.npy'.format(name[ind]), x)
        ind += 1

def load_set(sets):
    """Load existing data arrays from .npy files. Use if have preexisting data or when you don't to reshuffle the dataset"""
    return np.load('{}.npy'.format(sets))

In [ ]:
# below params assuming freq bins as channels
parameters={
    'k_height' : NUM_CHANNELS, #k_height, 
    'k_numFrames' : IMAGE_WIDTH, #k_numFrames, 
    'k_inputChannels' : IMAGE_HEIGHT, #k_inputChannels, 
    'K_NUMCONVLAYERS' : K_NUMCONVLAYERS, 
    'L1_CHANNELS' : L1_CHANNELS, 
    'L2_CHANNELS' : L2_CHANNELS, 
    'FC_SIZE' : FC_SIZE, 
    'K_ConvRows' : K_ConvRows, 
    'K_ConvCols' : K_ConvCols, 
    'k_ConvStrideRows' : k_ConvStrideRows, 
    'k_ConvStrideCols' : k_ConvStrideCols, 
    'k_poolRows' : k_poolRows, 
    'k_poolStrideRows' : k_poolStrideRows, 
    'k_downsampledHeight' : k_downsampledHeight, 
    'k_downsampledWidth' : k_downsampledWidth,
    'freqorientation' : FLAGS.freqorientation
}

In [ ]:
print(os.listdir("/home/huzaifah/Documents/compareTF/DataPrep/stft_png"))

In [29]:
NUM_THREADS = 4

# Will train data samples equal to either
#    nepochs * number of data records in the combined list of TFRecord files
#  or
#    k_nbatchsize * knbatches
#  which ever is smaller 
#
# Note: if you don't run for an integer number of epochs, some samples may be over/under represented

def getImage(fname, nepochs) :
    """ Reads data from the prepaired *list* files in fnames of TFRecords, does some preprocessing 
    params:
    fnames - list of filenames to read data from
    nepochs - An integer (optional). Just fed to tf.string_input_producer().  Reads through all data num_epochs times before generating an OutOfRange error. None means read forever.
    """

    label, image = spectreader.getImage(fname, nepochs)
    image = tf.reshape(image,[IMAGE_HEIGHT*IMAGE_WIDTH]) #same as np.flatten

    # re-define label as a "one-hot" vector 
    # it will be [0,1] or [1,0] here. 
    # This approach can easily be extended to more classes.
    label=tf.stack(tf.one_hot(label-1, N_LABELS))
    
    return label, image

def get_datafiles(a_dir, foldnumber):
    """ Returns a list of files in adir that start with the string in foldnumber """ 
    recordslist = []
    for i in foldnumber:
        recordslist.extend([a_dir + '/' + name for name in os.listdir(a_dir)
                if name.startswith(i)])
    return recordslist

In [31]:
target, data = getImage(get_datafiles(STFT_dataset_path, ['fold1','fold2','fold3','fold4']), k_nepochs)
imageBatch, labelBatch = tf.train.shuffle_batch(
    [data, target], batch_size=BATCH_SIZE,
    num_threads=NUM_THREADS,
    allow_smaller_final_batch=True, #want to finish an epoch even if datasize doesn't divide by batchsize
    enqueue_many=False, #IMPORTANT to get right, default=False - 
    capacity=1000,  #1000,
    min_after_dequeue=500) #500

vtarget, vdata = getImage(get_datafiles(STFT_dataset_path, ['fold5']), k_nepochs)
vimageBatch, vlabelBatch = tf.train.batch(
    [vdata, vtarget], batch_size=BATCH_SIZE,
    num_threads=NUM_THREADS,
    allow_smaller_final_batch=True, #want to finish an epoch even if datasize doesn't divide by batchsize
    enqueue_many=False, #IMPORTANT to get right, default=False - 
    capacity=1000)

getImage ['/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold1-00001-of-00002', '/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold1-00000-of-00002', '/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold2-00001-of-00002', '/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold2-00000-of-00002', '/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold3-00001-of-00002', '/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold3-00000-of-00002', '/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold4-00000-of-00002', '/home/huzaifah/Documents/compareTF/DataPrep/stft_png/fold4-00001-of-00002']
Tensor("one_hot_1:0", shape=(2,), dtype=float32)


In [ ]:
# Step 2: create placeholders for features (X) and labels (Y)
# each label is a one hot vector.
# 'None' here allows us to fill the placeholders with different size batches (which we do with training and validation batches)

X = tf.placeholder(tf.float32, [None,IMAGE_HEIGHT*IMAGE_WIDTH], name= "X")
x_image = tf.reshape(X, [-1,IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS]) 

# if FLAGS.freqorientation == "height" :
#     x_image = tf.reshape(x, [-1,k_height,k_numFrames,k_inputChannels]) 
# else :
#     print('set up reshaping for freqbins as channels')
#     foo1 = tf.reshape(x, [-1,k_freqbins,k_numFrames,1]) #unflatten (could skip this step if it wasn't flattenned in the first place!)
#     x_image = tf.transpose(foo1, perm=[0,3,2,1]) #moves freqbins from height to channel dimension

y = tf.placeholder(tf.float32, [None,N_LABELS], name= "Y")  #labeled classes, one-hot


### Now time to train and test the CNN

In [ ]:
# Path for tf.summary.FileWriter and to store model checkpoints
filewriter_path = save_path + "/filewriter/"
checkpoint_path = save_path + "/checkpoint/"

# Create parent path if it doesn't exist
if not os.path.isdir(checkpoint_path): os.mkdir(checkpoint_path)

#********************************************************************

# tf Graph input placeholders
x = tf.placeholder(tf.float32, [BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS])
y = tf.placeholder(tf.int32, [None, N_LABELS])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

# Construct model
pred = m.conv_net(x, m.weights, m.biases, keep_prob)

#L2 regularization
lossL2 = tf.add_n([tf.nn.l2_loss(val) for name,val in m.weights.items()]) * beta #L2 reg on all weight layers
lossL2_onlyfull = tf.add_n([tf.nn.l2_loss(m.weights['wd1']),tf.nn.l2_loss(m.weights['out'])]) * beta #L2 reg on dense layers

# Op for calculating the loss
with tf.name_scope("cross_ent"):
    if l2reg:
        if l2regfull:
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y) + lossL2_onlyfull)
        else:
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y) + lossL2)
    else:
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# Train op
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(epsilon=epsilon).minimize(loss)

# Add the loss to summary
tf.summary.scalar('cross_entropy', loss)

# Predictions
prob = tf.nn.softmax(pred)

# Evaluation op: Accuracy of the model
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(prob, 1), tf.argmax(y, 1))
    accuracy = 100*tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Add the accuracy to the summary
tf.summary.scalar('accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

# Initialize an saver for store model checkpoints
saver = tf.train.Saver()

In [ ]:
max_acc = []
max_epochs = []
fold_list = [x+1 for x in range(max(folds))]

total_runs = TOTAL_RUNS

start_time_long = time.monotonic()
text_file = open(save_path + "/stft-double_v2.txt", "w") #save training data
print("{} Open Tensorboard at --logdir {}".format(datetime.now(), filewriter_path))

run = 0
while run < total_runs:
    for i in fold_list:

        test_acc_list = []

        train_set = images[folds!=i] #first get the right fold in order of fold_list
        train_label = labels[folds!=i]
        test_set = images[folds==i]
        test_label = labels[folds==i]

        text_file.write('*** Initializing fold #%u as test set ***\n' % i)
        print('*** Initializing fold #%u as test set ***' % i)

        train_batches_per_epoch, train_extra = divmod(train_label.shape[0],BATCH_SIZE)
        test_batches_per_epoch, test_extra = divmod(test_label.shape[0],BATCH_SIZE)

        # Initialize the FileWriter
        writer = tf.summary.FileWriter(filewriter_path + str(i))

        with tf.Session() as session:

            # Initialize all variables        
            tf.global_variables_initializer().run()

            # Add the model graph to TensorBoard
            writer.add_graph(session.graph)

            print("{} Start training...".format(datetime.now()))

            start_time = time.monotonic()

            for e in range(epochs):
                print("{} Epoch number: {}".format(datetime.now(), e+1))
                text_file.write("{} Epoch number: {}\n".format(datetime.now(), e+1))
                
                #shuffle order every epoch
                train_set, train_label = shuffle(train_set, train_label)
                test_set, test_label = shuffle(test_set, test_label)

                step = 1

                while step < train_batches_per_epoch:
                    #create training mini-batch here
                    offset = (step * BATCH_SIZE) % (train_label.shape[0] - BATCH_SIZE)
                    batch_data = train_set[offset:(offset + BATCH_SIZE), :, :, :]
                    batch_labels = train_label[offset:(offset + BATCH_SIZE),:]
                    
                    #train and backprop
                    session.run(optimizer, feed_dict= {x:batch_data, y:batch_labels, keep_prob:dropout})
                    
                    #run merged_summary to display progress on Tensorboard
                    if (step % display_step == 0):               
                        s = session.run(merged_summary, feed_dict={x: batch_data, y: batch_labels, keep_prob: 1.})
                        writer.add_summary(s, e*train_batches_per_epoch + step) 
                    step += 1

                test_acc = 0.
                test_count = 0
                for i in range(test_batches_per_epoch):
                    #prepare test mini-batch
                    offset = (i * BATCH_SIZE) % (test_label.shape[0] - BATCH_SIZE)
                    test_batch = test_set[offset:(offset + BATCH_SIZE), :, :, :]
                    label_batch = test_label[offset:(offset + BATCH_SIZE),:]

                    acc = session.run(accuracy, feed_dict={x: test_batch, y: label_batch, keep_prob: 1.})

                    test_acc += acc*BATCH_SIZE
                    test_count += 1*BATCH_SIZE
                
                #sometimes we get leftovers if batch_size is not a factor of the training/test set
                if test_extra != 0:
                    test_batch = test_set[-test_extra,:,:,:]
                    label_batch = test_label[-test_extra:,:]
                    acc = session.run([accuracy,prob], feed_dict={x: test_batch, y: label_batch, keep_prob: 1.})

                    test_acc += acc*test_extra
                    test_count += 1*test_extra
                
                #calculate total test accuracy
                test_acc /= test_count 
                print("{} Test Accuracy = {:.4f}".format(datetime.now(),test_acc))
                text_file.write("{} Test Accuracy = {:.4f}\n".format(datetime.now(),test_acc))
                test_acc_list.append(test_acc)

                #save checkpoint of the model
                if ((e+1) % checkpoint_epoch == 0):  
                    checkpoint_name = os.path.join(checkpoint_path, dataset_name+'model_fold'+i+'_epoch'+str(e+1)+'.ckpt')
                    save_path = saver.save(session, checkpoint_name)  
                    print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))

            # find the max test score and the epoch it belongs to        
            max_acc.append(max(test_acc_list))
            max_epoch = test_acc_list.index(max(test_acc_list))
            max_epochs.append(max_epoch) 

            elapsed_time = time.monotonic() - start_time
            text_file.write("--- Training time taken: {} ---\n".format(time_taken(elapsed_time)))
            print("--- Training time taken:",time_taken(elapsed_time),"---")
            print("------------------------")
        
        # return the max accuracies of each fold and their respective epochs
        print(max_acc)
        print(max_epochs)
    run += 1

elapsed_time_long = time.monotonic() - start_time_long
print("*** All runs completed ***")
text_file.write("Total time taken:")
text_file.write(time_taken(elapsed_time_long))
print("Total time taken:",time_taken(elapsed_time_long))
text_file.close()